# 초기 가중치 저장 및 모델 생성

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D


def init_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(299, 299, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4, activation='softmax'))
    return model


def make_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', input_shape=(299, 299, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=(3, 3), padding="same", activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(4, activation='softmax'))
    return model


model = make_model()
model_json = model.to_json()
with open("init_grayscale_model.json", "w") as json_file : 
    json_file.write(model_json)
print("Save init_model")

model = init_model()    
model.save_weights("init_grayscale_weight.h5")
print("Saved init_weight")

Save init_model
Saved init_weight


# Total batch1 train & evaluation

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json 

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'D:/OCT-Image/total1/train',
        target_size=(299, 299),
        color_mode = 
        batch_size=64,                                
        class_mode='categorical')

json_file = open("init_model.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
model1 = model_from_json(loaded_model_json)

model1.load_weights("init_weight.h5")
model1.compile(loss='categorical_crossentropy',
                         optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                         metrics=['acc'])

model1.summary()

print("start train")
model1.fit_generator(train_generator, epochs = 30)  # 학습시작 
print("train done")

Using TensorFlow backend.


Found 76484 images belonging to 4 classes.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 299, 299, 32)      896       
_________________________________________________________________
batch_normalization_18 (Batc (None, 299, 299, 32)      128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 149, 149, 64)      18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 149, 149, 64)      256       
_________________________________________________________________
max_pooling

In [2]:
import numpy as np
from PIL import Image
import glob

test_path = "D:/OCT-Image/total1/test/"
categories = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

image_w = 299
image_h = 299

for idx, categorie in enumerate(categories):
    result_list = []
    cnv = []
    dme = []
    drusen = []
    normal = []
    files = glob.glob(test_path + categorie + "/*")
    
    
    for idx2, file in enumerate(files):
        img = Image.open(file)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        data = data.reshape((1,image_w,image_h,3))
        data = data / 255
        result = model1.predict_classes(data)
        
        if result[0] == 0:
            cnv.append(result[0])
        elif result[0] == 1:
            dme.append(result[0])
        elif result[0] == 2:
            drusen.append(result[0])
        else:
            normal.append(result[0])

        
        if categorie == 'CNV'and result[0] == 0:
            result_list.append(1)
        elif categorie == 'DME' and result[0] == 1:
            result_list.append(1)
        elif categorie == 'DRUSEN' and result[0] == 2:
            result_list.append(1)
        elif categorie == 'NORMAL' and result[0] == 3:
            result_list.append(1)
        else:
            result_list.append(0)
    
    
    
    print("CNV :"  + str(len(cnv)) + "\nDME :" + str(len(dme)) + "\nDrusen :" + str(len(drusen)) + "\nNormal :" + str(len(normal)))
    print(result_list)
    print(categorie + "의 test정확도 : " + str(np.mean(result_list)))

CNV :1705
DME :229
Drusen :2
Normal :64
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

CNV :631
DME :247
Drusen :134
Normal :988
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Total batch2 train & evaluation

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json 

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'D:/OCT-Image/total2/train',
        target_size=(299, 299),                       
        batch_size=64,                                
        class_mode='categorical')

json_file = open("init_model.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
model1 = model_from_json(loaded_model_json)

model1.load_weights("init_weight.h5")
model1.compile(loss='categorical_crossentropy',
                         optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                         metrics=['acc'])

model1.summary()

print("start train")
model1.fit_generator(train_generator, epochs = 30)  # 학습시작 
print("train done")

Using TensorFlow backend.


Found 76484 images belonging to 4 classes.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 299, 299, 32)      896       
_________________________________________________________________
batch_normalization_18 (Batc (None, 299, 299, 32)      128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 149, 149, 64)      18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 149, 149, 64)      256       
_________________________________________________________________
max_pooling

In [2]:
import numpy as np
from PIL import Image
import glob

test_path = "D:/OCT-Image/total2/test/"
categories = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

image_w = 299
image_h = 299

for idx, categorie in enumerate(categories):
    result_list = []
    cnv = []
    dme = []
    drusen = []
    normal = []
    files = glob.glob(test_path + categorie + "/*")
    
    
    for idx2, file in enumerate(files):
        img = Image.open(file)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        data = data.reshape((1,image_w,image_h,3))
        data = data / 255
        result = model1.predict_classes(data)
        
        if result[0] == 0:
            cnv.append(result[0])
        elif result[0] == 1:
            dme.append(result[0])
        elif result[0] == 2:
            drusen.append(result[0])
        else:
            normal.append(result[0])

        
        if categorie == 'CNV'and result[0] == 0:
            result_list.append(1)
        elif categorie == 'DME' and result[0] == 1:
            result_list.append(1)
        elif categorie == 'DRUSEN' and result[0] == 2:
            result_list.append(1)
        elif categorie == 'NORMAL' and result[0] == 3:
            result_list.append(1)
        else:
            result_list.append(0)
    
    
    
    print("CNV :"  + str(len(cnv)) + "\nDME :" + str(len(dme)) + "\nDrusen :" + str(len(drusen)) + "\nNormal :" + str(len(normal)))
    print(result_list)
    print(categorie + "의 test정확도 : " + str(np.mean(result_list)))

CNV :1852
DME :96
Drusen :37
Normal :15
[1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

CNV :504
DME :129
Drusen :579
Normal :788
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 

# Total batch3 train & evaluation

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json 

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'D:/OCT-Image/total3/train',
        target_size=(299, 299),                       
        batch_size=64,                                
        class_mode='categorical')

json_file = open("init_model.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
model1 = model_from_json(loaded_model_json)

model1.load_weights("init_weight.h5")
model1.compile(loss='categorical_crossentropy',
                         optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                         metrics=['acc'])

model1.summary()

print("start train")
model1.fit_generator(train_generator, epochs = 30)  # 학습시작 
print("train done")

Using TensorFlow backend.


Found 76484 images belonging to 4 classes.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 299, 299, 32)      896       
_________________________________________________________________
batch_normalization_18 (Batc (None, 299, 299, 32)      128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 149, 149, 64)      18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 149, 149, 64)      256       
_________________________________________________________________
max_pooling

In [2]:
import numpy as np
from PIL import Image
import glob

test_path = "D:/OCT-Image/total3/test/"
categories = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

image_w = 299
image_h = 299

for idx, categorie in enumerate(categories):
    result_list = []
    cnv = []
    dme = []
    drusen = []
    normal = []
    files = glob.glob(test_path + categorie + "/*")
    
    
    for idx2, file in enumerate(files):
        img = Image.open(file)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        data = data.reshape((1,image_w,image_h,3))
        data = data / 255
        result = model1.predict_classes(data)
        
        if result[0] == 0:
            cnv.append(result[0])
        elif result[0] == 1:
            dme.append(result[0])
        elif result[0] == 2:
            drusen.append(result[0])
        else:
            normal.append(result[0])

        
        if categorie == 'CNV'and result[0] == 0:
            result_list.append(1)
        elif categorie == 'DME' and result[0] == 1:
            result_list.append(1)
        elif categorie == 'DRUSEN' and result[0] == 2:
            result_list.append(1)
        elif categorie == 'NORMAL' and result[0] == 3:
            result_list.append(1)
        else:
            result_list.append(0)
    
    
    
    print("CNV :"  + str(len(cnv)) + "\nDME :" + str(len(dme)) + "\nDrusen :" + str(len(drusen)) + "\nNormal :" + str(len(normal)))
    print(result_list)
    print(categorie + "의 test정확도 : " + str(np.mean(result_list)))

CNV :1215
DME :577
Drusen :56
Normal :152
[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

CNV :425
DME :528
Drusen :232
Normal :815
[0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Total batch4 train & evaluation

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json 

train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        'D:/OCT-Image/total4/train',
        target_size=(299, 299),                       
        batch_size=64,                                
        class_mode='categorical')

json_file = open("init_model.json", "r") 
loaded_model_json = json_file.read() 
json_file.close() 
model1 = model_from_json(loaded_model_json)

model1.load_weights("init_weight.h5")
model1.compile(loss='categorical_crossentropy',
                         optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
                         metrics=['acc'])

model1.summary()

print("start train")
model1.fit_generator(train_generator, epochs = 30)  # 학습시작 
print("train done")

Using TensorFlow backend.


Found 76484 images belonging to 4 classes.
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 299, 299, 32)      896       
_________________________________________________________________
batch_normalization_18 (Batc (None, 299, 299, 32)      128       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 149, 149, 32)      0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 149, 149, 32)      0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 149, 149, 64)      18496     
_________________________________________________________________
batch_normalization_19 (Batc (None, 149, 149, 64)      256       
_________________________________________________________________
max_pooling

In [2]:
import numpy as np
from PIL import Image
import glob

test_path = "D:/OCT-Image/total4/test/"
categories = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

image_w = 299
image_h = 299

for idx, categorie in enumerate(categories):
    result_list = []
    cnv = []
    dme = []
    drusen = []
    normal = []
    files = glob.glob(test_path + categorie + "/*")
    
    
    for idx2, file in enumerate(files):
        img = Image.open(file)
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        data = data.reshape((1,image_w,image_h,3))
        data = data / 255
        result = model1.predict_classes(data)
        
        if result[0] == 0:
            cnv.append(result[0])
        elif result[0] == 1:
            dme.append(result[0])
        elif result[0] == 2:
            drusen.append(result[0])
        else:
            normal.append(result[0])

        
        if categorie == 'CNV'and result[0] == 0:
            result_list.append(1)
        elif categorie == 'DME' and result[0] == 1:
            result_list.append(1)
        elif categorie == 'DRUSEN' and result[0] == 2:
            result_list.append(1)
        elif categorie == 'NORMAL' and result[0] == 3:
            result_list.append(1)
        else:
            result_list.append(0)
    
    
    
    print("CNV :"  + str(len(cnv)) + "\nDME :" + str(len(dme)) + "\nDrusen :" + str(len(drusen)) + "\nNormal :" + str(len(normal)))
    print(result_list)
    print(categorie + "의 test정확도 : " + str(np.mean(result_list)))

CNV :1724
DME :188
Drusen :18
Normal :70
[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1

CNV :984
DME :227
Drusen :104
Normal :685
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 